In [11]:
using Turing
using LimberJack
using PythonCall
using NPZ;

In [12]:
np = pyimport("numpy");

In [13]:
meta = np.load("../data/FD/K1K_DELS_DESY1_eBOSS_meta.npz")
tracers_names = pyconvert(Vector{String}, meta["tracers"])
pairs = pyconvert(Vector{Vector{String}}, meta["pairs"])
pairs_ids = pyconvert(Vector{Vector{Int}}, meta["pairs_ids"])
idx = pyconvert(Vector{Int}, meta["idx"])
data_vector = pyconvert(Vector{Float64}, meta["cls"])
cov_tot = pyconvert(Matrix{Float64}, meta["cov"]);

In [44]:
#files = npzread("../data/FD/K1K_DELS_DESY1_eBOSS_files.npz");
for m in eachrow(files["nz_DECALS__0_0"])
    println(length(m))
end

385
385


In [53]:
A = files["nz_DECALS__0_0"]
B = [A[i,:] for i in 1:size(A,1)]
B[2]

385-element Vector{Float64}:
 0.10312656219913398
 0.10902732139389724
 0.1155060288488159
 0.12264680481658076
 0.13054908210601449
 0.1393301594051586
 0.1491277396632761
 0.1601022189350765
 0.17243847957611974
 0.1863469157882907
 0.2020632281754137
 0.21984608690732854
 0.23997130698863756
 ⋮
 0.0021773375625269774
 0.002158390728064383
 0.002139674813198292
 0.0021211861964579265
 0.002102921325845262
 0.002084876717258931
 0.0020670489529593934
 0.0020494346800741608
 0.0020320306091418895
 0.0020148335126941898
 0.0019978402238740533
 0.001981047635089811

In [27]:
files["ls_DECALS__0_0_DECALS__0_0"]

3-element Vector{Float64}:
  44.500000000000014
  74.50000000000001
 104.50000000000003

In [15]:
function make_cls(pars)
    nuisances = Dict("DESgc__0_0_b" => pars[5],
                     "DESgc__1_0_b" => pars[6],
                     "DESgc__2_0_b" => pars[7],
                     "DESgc__3_0_b" => pars[8],
                     "DESgc__4_0_b" => pars[9],
                     "DESgc__0_0_dz" => pars[10],
                     "DESgc__1_0_dz" => pars[11],
                     "DESgc__2_0_dz" => pars[12],
                     "DESgc__3_0_dz" => pars[13],
                     "DESgc__4_0_dz" => pars[14],

                     "DESgc__0_e_dz" => pars[21],
                     "DESgc__1_e_dz" => pars[22],
                     "DESgc__2_e_dz" => pars[23],
                     "DESgc__3_e_dz" => pars[24],
                     "DESgc__0_e_m" => pars[17],
                     "DESgc__1_e_m" => pars[18],
                     "DESgc__2_e_m" => pars[19],
                     "DESgc__3_e_m" => pars[20],

                     "A_IA" => pars[15],
                     "alpha_IA" => pars[16],
        
                     "DECALS__0_0_b" => 1.0,
                     "DECALS__1_0_b" => 1.0,
                     "DECALS__2_0_b" => 1.0,
                     "DECALS__3_0_b" => 1.0,
                     "DECALS__0_0_dz" => 0.0,
                     "DECALS__1_0_dz" => 0.0,
                     "DECALS__2_0_dz" => 0.0,
                     "DECALS__3_0_dz" => 0.0,
                    
                     "eBOSS__0_0_b" => 1.0,
                     "eBOSS__1_0_b" => 1.0,
                    
                     "KiDS1000__0_e_dz" => 0.0,
                     "KiDS1000__1_e_dz" => 0.0,
                     "KiDS1000__2_e_dz" => 0.0,
                     "KiDS1000__3_e_dz" => 0.0,
                     "KiDS1000__0_e_m" => 0.0,
                     "KiDS1000__1_e_m" => 0.0,
                     "KiDS1000__2_e_m" => 0.0,
                     "KiDS1000__3_e_m" => 0.0)

    Ωm = pars[26]
    Ωb = pars[1]
    h = pars[4]
    ns = pars[3]
    s8 = pars[25]
    cosmology = LimberJack.Cosmology(Ωm, Ωb, h, ns, s8, 
                                     tk_mode="emulator", Pk_mode="Halofit")

    return Theory(cosmology,
                  tracers_names, pairs,
                  pairs_ids, idx, files;
                  nz_path=nz_path,
                  Nuisances=nuisances)

end

make_cls (generic function with 1 method)

In [16]:
@model function model(data_vector::Vector{Float64})
    Ωm ~ Uniform(0.1, 0.6)
    Ωb = 0.05 #~ Uniform(0.03, 0.07)
    h = 0.67 #~ Uniform(0.6, 0.9)
    s8 = 0.81 #~ Uniform(0.6, 1.0)
    ns = 0.96 #~ Uniform(0.87, 1.07)

    cosmology = LimberJack.Cosmology(Ωm, Ωb, h, s8, s8,
                                     tk_mode="emulator",
                                     Pk_mode="Halofit")
    theory = Theory(cosmology,
                    tracers_names, pairs,
                    pairs_ids, idx, files)
    data_vector ~ MvNormal(theory, cov_tot)
    return data_vector
end;

In [17]:
iterations = 300
nadapts = 100;

In [18]:
chain1 = sample(model(data_vector), MH(), iterations,
            nadapt=nadapts, progress=true; save_state=true);

LoadError: KeyError: key "nz_DECALS__0_0.npz" not found

In [34]:
chain3 = sample(model(data_vector), NUTS(5, 0.1; init_ϵ=0.03), iterations,
                progress=true; save_state=true);

┌ Warning: The current proposal will be rejected due to numerical error(s).
│   isfinite.((θ, r, ℓπ, ℓκ)) = (true, false, false, false)
└ @ AdvancedHMC /home/jaime/.julia/packages/AdvancedHMC/MIxdK/src/hamiltonian.jl:47
┌ Warning: The current proposal will be rejected due to numerical error(s).
│   isfinite.((θ, r, ℓπ, ℓκ)) = (true, false, false, false)
└ @ AdvancedHMC /home/jaime/.julia/packages/AdvancedHMC/MIxdK/src/hamiltonian.jl:47
┌ Warning: The current proposal will be rejected due to numerical error(s).
│   isfinite.((θ, r, ℓπ, ℓκ)) = (true, false, false, false)
└ @ AdvancedHMC /home/jaime/.julia/packages/AdvancedHMC/MIxdK/src/hamiltonian.jl:47
Sampling:   1%|▌                                        |  ETA: 0:03:59┌ Warning: The current proposal will be rejected due to numerical error(s).
│   isfinite.((θ, r, ℓπ, ℓκ)) = (true, false, false, false)
└ @ AdvancedHMC /home/jaime/.julia/packages/AdvancedHMC/MIxdK/src/hamiltonian.jl:47
Sampling:   2%|▊                                 

390
390
390
390
390
390
390
390
390
390
390
390
390
390
390
390
390
390
390
390
390
390
390
390
390
390
390
390
390
390
390
390
390
390
390
390
390
390
390
390
390
390
390
390
390
390
390
390
390
390
390
390
390
390
390
390
390
390
390
390
390
390
390
390
390
390
390
390
390
390
390
390
390
390
390
390
390
390
390
390
390
390
390
390
390
390
390
390
390
390
390
390


LoadError: InterruptException: